In [1]:
import subprocess
from urllib.parse import urlparse
import os
import boto3


In [ ]:
def execute(args):
    print('running command : %s' % ( ' '.join(args) ))
    process = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    output = process.communicate()
    print ('STDOUT:{}'.format(output))


In [2]:
def split_s3_path(s3_path):
    o = urlparse(s3_path.strip())
    print(o)
    return o.netloc, o.path[1:]

In [3]:
def generate_data():
    
    path = "/Users/nikki/work/code/Fake-Apache-Log-Generator/" 
    os.chdir(path)
    
    cmd = [ "./venv/bin/python", "apache-fake-log-gen.py", "-n", "1000", "-o", "GZ"]
    execute(cmd)
    
    # List files in current directory
    files = os.listdir(os.curdir) 
    # particular files from a directories
    list = [path + k for k in files if 'access_log' in k]
    print(list)
    return list

In [4]:
def del_local_file(list_of_file):
    for file in list_of_file:
        print("Removing file from local")
        os.remove(file)


In [5]:
def copy_to_s3(dest_path, log_files):
    bucket, path = split_s3_path(dest_path)
    print('bucket : %s and path : %s '%( bucket, path))
    s3 = boto3.resource('s3')
    bkt = s3.Bucket(bucket)
    
    for log in log_files:
        filename = log.split("/")[-1]
        print('Copying file %s to S3 loaction %s' % (log,path+filename))
        bkt.upload_file(log,path+filename)
    

In [6]:
def run(date,location):
    log_files = generate_data()

    copy_to_s3(location + date + '/', log_files)

    del_local_file(log_files)

In [7]:
date = '2018-01-03'
location = 's3://nikita-ds-playground/raw/access-log/'

run(date,location)


running command : ./venv/bin/python apache-fake-log-gen.py -n 1000 -o GZ
STDOUT:(b'', None)
['/Users/nikki/work/code/Fake-Apache-Log-Generator/access_log_20181229-170053.log.gz']
ParseResult(scheme='s3', netloc='nikita-ds-playground', path='/raw/access-log/2018-01-03/', params='', query='', fragment='')
bucket : nikita-ds-playground and path : raw/access-log/2018-01-03/ 
Copying file /Users/nikki/work/code/Fake-Apache-Log-Generator/access_log_20181229-170053.log.gz to S3 loaction raw/access-log/2018-01-03/access_log_20181229-170053.log.gz
Removing file from local
